In [2]:
# Pain Level Detection with Full Body Language Analysis
# Real-Time Implementation with YOLOv8 Pose Tracking

import cv2
import numpy as np
import tensorflow as tf
from ultralytics import YOLO
from collections import defaultdict, deque
from IPython.display import display, clear_output
from PIL import Image

In [3]:
# ----------------------------
# 1. Configuration & Constants
# ----------------------------
PAIN_LEVELS = {
    0: {'name': 'No Pain', 'color': (0, 255, 0), 'threshold': 0.7},
    1: {'name': 'Low Pain', 'color': (0, 255, 255), 'threshold': 0.65},
    2: {'name': 'Medium Pain', 'color': (0, 165, 255), 'threshold': 0.6},
    3: {'name': 'High Pain', 'color': (0, 0, 255), 'threshold': 0.55},
    4: {'name': 'Unbearable Pain', 'color': (0, 0, 128), 'threshold': 0.5}
}

SEQ_LENGTH = 45  # 1.5 seconds at 30 FPS
KEYPOINT_INDICES = {
    'nose': 0,
    'shoulders': [5, 6],    # Left and right shoulders
    'hips': [11, 12],       # Left and right hips
    'knees': [13, 14],      # Left and right knees
    'elbows': [7, 8]        # Left and right elbows
}
INPUT_SHAPE = (SEQ_LENGTH, 17*3)  # 17 keypoints (x, y, confidence)


In [4]:
# ----------------------------
# 2. Initialize YOLOv8 Models
# ----------------------------
pose_model = YOLO('yolov8n-pose.pt')  # Pose estimation model

In [5]:
# ----------------------------
# 3. Pain Detection Model
# ----------------------------
class PainDetector(tf.keras.Model):
    def __init__(self):
        super(PainDetector, self).__init__()
        self.lstm1 = tf.keras.layers.LSTM(128, return_sequences=True)
        self.dropout1 = tf.keras.layers.Dropout(0.3)
        self.lstm2 = tf.keras.layers.LSTM(64)
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dropout2 = tf.keras.layers.Dropout(0.4)
        self.output_layer = tf.keras.layers.Dense(5, activation='softmax')
        
    def call(self, inputs):
        x = self.lstm1(inputs)
        x = self.dropout1(x)
        x = self.lstm2(x)
        x = self.dense1(x)
        x = self.dropout2(x)
        return self.output_layer(x)

pain_model = PainDetector()
pain_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
# ----------------------------
# 4. Body Language Feature Extraction
# ----------------------------
def extract_pain_features(keypoints):
    """Extract biomechanical features based on described pain characteristics"""
    features = []
    
    # Shoulder tension (Low/Medium Pain)
    left_shoulder = keypoints[KEYPOINT_INDICES['shoulders'][0]*3:(KEYPOINT_INDICES['shoulders'][0]*3)+2]
    right_shoulder = keypoints[KEYPOINT_INDICES['shoulders'][1]*3:(KEYPOINT_INDICES['shoulders'][1]*3)+2]
    shoulder_distance = np.linalg.norm(left_shoulder - right_shoulder)
    features.append(shoulder_distance)
    
    # Hip flexibility (Medium/High Pain)
    left_hip = keypoints[KEYPOINT_INDICES['hips'][0]*3:(KEYPOINT_INDICES['hips'][0]*3)+2]
    right_hip = keypoints[KEYPOINT_INDICES['hips'][1]*3:(KEYPOINT_INDICES['hips'][1]*3)+2]
    hip_angle = np.arctan2(right_hip[1]-left_hip[1], right_hip[0]-left_hip[0])
    features.append(hip_angle)
    
    # Spinal curvature (High/Unbearable Pain)
    nose = keypoints[KEYPOINT_INDICES['nose']*3:(KEYPOINT_INDICES['nose']*3)+2]
    spine_curvature = np.mean([np.linalg.norm(nose - left_hip), np.linalg.norm(nose - right_hip)])
    features.append(spine_curvature)
    
    # Movement jerkiness (All pain levels)
    movement_smoothness = np.mean(np.abs(np.diff(keypoints[::3])))  # X coordinates
    features.append(movement_smoothness)
    
    return np.array(features)


In [10]:
# ----------------------------
# 5. Real-Time Processing
# ----------------------------
def analyze_body_language():
    cap = cv2.VideoCapture(0)
    track_history = defaultdict(lambda: {'queue': deque(maxlen=SEQ_LENGTH), 'last_prediction': 0})
    
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            continue
            
        # Process frame with YOLOv8
        results = pose_model.track(frame, persist=True, verbose=False)
        annotated_frame = results[0].plot()
        
        if results[0].keypoints is not None:
            for box_id, kps in enumerate(results[0].keypoints.xy.cpu().numpy()):
                # Get tracking ID
                track_id = box_id if results[0].boxes.id is None else results[0].boxes.id[box_id].item()
                
                # Process keypoints
                keypoints = kps.flatten()
                track_history[track_id]['queue'].append(keypoints)
                
                # Analyze when buffer full
                if len(track_history[track_id]['queue']) == SEQ_LENGTH:
                    # Convert to sequence array
                    sequence = np.array(track_history[track_id]['queue'])
                    
                    # Normalize and predict
                    sequence_norm = (sequence - np.mean(sequence)) / np.std(sequence)
                    prediction = pain_model.predict(np.expand_dims(sequence_norm, 0), verbose=0)[0]
                    
                    # Store prediction
                    track_history[track_id]['last_prediction'] = np.argmax(prediction)
                    confidence = np.max(prediction)
                    
                    # Get pain level info
                    pain_info = PAIN_LEVELS[track_history[track_id]['last_prediction']]
                    
                    # Draw annotations
                    text = f"ID {track_id}: {pain_info['name']} ({confidence:.2f})"
                    cv2.putText(annotated_frame, text, (10, 30 + (box_id * 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, pain_info['color'], 2))
                    
                    # Add warning for high pain levels
                    if track_history[track_id]['last_prediction'] >= 3:
                        cv2.putText(annotated_frame, "MEDICAL ATTENTION NEEDED!", 
                                   (frame.shape[1]-400, 30 + (box_id * 30)),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # Display frame with OpenCV
        cv2.imshow('Pain Level Detection', annotated_frame)
        
        # Exit on 'q' press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

# ----------------------------
# 6. Run the System
# ----------------------------
# Load pre-trained weights (replace with actual trained weights)
# pain_model.load_weights('pain_detection_model_weights.h5')

# Start real-time analysis
analyze_body_language()
